In [3]:
%reload_ext autoreload
%autoreload 2

from func import *
import pandas as pd



In [4]:
filename = 'all_data_anonymized.csv'
date_col = 'todate'
start_date = '1980-01-01'
seperator = ';'
fill = 0
lim = 5
df_read = load_and_preprocess_data(filename, date_col, start_date, seperator, fill, lim)


c:\Users\osaz001\Master-thesis\Files\func.py:21: DtypeWarning: Columns (15,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_read = pd.read_csv(filename, sep=seperator)


In [5]:
windows = [5, 10, 20, 40, 60, 100, 180, 240, 360, 480]
window_m = [10, 30, 60, 100, 180]
assets = df_read.columns
df_feat = add_features(df_read, window_m)

In [6]:
target_days = 9

return_column_shift = 'avgreturn'
volatility_column_shift = 'volatility'
df = add_y_col(df_feat, df_read, date_col, target_days, return_column_shift, volatility_column_shift)

In [7]:
X_train, X_eval, X_test, y_train, y_eval, y_test, X_train_eval, y_train_eval = prepare_training_dataset(df, date_col, shuffle=False, train_split=0.25, eval_split=0.25)

In [8]:
param_grid_alpha = {'alpha': [0.1, 1.0, 10.0]}
ridge_best, grid_search = optimize_and_train_ridge(X_train, y_train, X_train_eval, y_train_eval, param_grid_alpha)

Best parameters: {'alpha': 10.0}
Best accuracy on evaluation set: 0.5086063469241039


In [9]:
param_grid_xgb = {
    'max_depth': [3, 6, 10],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2]
}

xgb_best, best_params = optimize_and_train_xgb(X_train, y_train, X_eval, y_eval, param_grid_xgb)


c:\appl\Anaconda3\lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}


In [ ]:

param_grid_lgb = {
    'max_depth': [3, 6, 10],
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [31, 63, 127, 255]
}

# Call the function with your datasets and hyperparameter grid
lgb_best, best_params = optimize_and_train_lgb(X_train, y_train, X_eval, y_eval, param_grid_lgb)

In [ ]:
NN_model, history, X_test_scaled = train_and_evaluate_NN(X_train_eval, y_train_eval, X_eval, y_eval, X_test, y_test)


In [ ]:

# Store predictions in a dictionary
predictions = {
    "Ridge Classifier": ridge_best.predict(X_test),
    "XGBoost Classifier": xgb_best.predict(X_test),
    "LightGBM Classifier": lgb_best.predict(X_test),
}   

# Iterate through the dictionary and evaluate each model
for model_name, y_pred in predictions.items():
    print(model_name + ":")
    evaluate_model_performance(y_test, y_pred)
    print("\n")


In [ ]:
X_test.shape

In [18]:
%reload_ext autoreload
%autoreload 2
models = {
    'xgb': xgb_best,
    'ridge': ridge_best
}

# Dictionary to hold the results
results_best = {}
results_worst = {}

for name, model in models.items():
    results_best[name], results_worst[name] = predict_and_analyze_ext(model, X_test, df, name) 

In [21]:
# keep same index from X_test in df as new df

df_2 = df.loc[X_test.index]


In [30]:
dddddd = df[['todate']]
dddddd.head()

,todate
120284,1997-07-25
120285,1997-07-25
120286,1997-07-25
120287,1997-07-25
120288,1997-07-25


In [22]:

# Use the dates in df_2 to filter rows in df_read
df_2_read = df_read.loc[df_2[date_col].unique()]

# fill missing values with 0
df_2_read.fillna(0, inplace=True)

In [23]:
import pandas as pd

df_2_neutral = df_2_read.copy()

for col in df_2_neutral.columns:
    df_2_neutral[col] = 0  


model_dfs = {
    'xgb': df_2_neutral.copy(),
    'ridge': df_2_neutral.copy()
}



NameError: name 'a' is not defined

In [14]:
ridge_best.predict(X_test.iloc[1])

c:\appl\Anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RidgeClassifier was fitted with feature names
  warnings.warn(


ValueError: Expected 2D array, got 1D array instead:
array=[-6.88621300e-03 -1.32046382e-02 -1.55207620e-02 -6.89701700e-03
 -1.54267930e-02  1.60888967e-03  8.54843907e-04  4.50969017e-04
  1.05358157e-03 -3.10840138e-04  1.44800070e-02  8.29198590e-02
  7.89195780e-02  2.95002840e-02 -1.80287280e-02 -4.03750512e-01
 -4.18091379e-01 -6.57602500e-01  3.63896731e-01 -3.52495334e-01
  6.00431533e-03  8.22148406e-03  9.06711178e-03  5.64395053e-03
  8.83360659e-03 -4.19803530e-02 -3.69427000e-03  1.02978780e-02
  4.23327600e-03 -8.69978500e-03 -2.72703880e-02  3.85981530e-02
 -8.30574100e-03  2.40736000e-02].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [24]:
def update_df_with_asset_performance(df, portfolios, value):
    """ Update df by marking assets in portfolios with a specified value. """
    for date, assets in portfolios.items():
        for asset in assets:
            if asset in df.columns: 
                df.at[pd.to_datetime(date), asset] = value


# Gather all best and worst assets for each date for each model
portfolios_best = { 'lgb': {}, 'xgb': {}, 'ridge': {} }
portfolios_worst = { 'lgb': {}, 'xgb': {}, 'ridge': {} }

for name, model in models.items():
    best_assets, worst_assets = predict_and_analyze_ext(model, X_test, df, name) 

    # Accumulate best and worst assets for each date
    for date in best_assets['todate'].unique():
        portfolios_best[name].setdefault(date, []).extend(best_assets[best_assets['todate'] == date]['asset'].tolist())
        portfolios_worst[name].setdefault(date, []).extend(worst_assets[worst_assets['todate'] == date]['asset'].tolist())

    update_df_with_asset_performance(model_dfs[name], portfolios_best[name], 1)
    update_df_with_asset_performance(model_dfs[name], portfolios_worst[name], -1)




In [25]:
a = model_dfs['ridge']


b = df_2_read

ab = a*b
#np.dot(a,b).cumsum()

In [26]:
X_test.head()

,VaR_10,VaR_100,VaR_180,VaR_30,VaR_60,avgreturn_10,avgreturn_100,avgreturn_180,avgreturn_30,avgreturn_60,momentum_10,momentum_100,momentum_180,momentum_30,momentum_60,skew_10,skew_100,skew_180,skew_30,skew_60,volatility_10,volatility_100,volatility_180,volatility_30,volatility_60,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
319106,-0.009637,-0.012813,-0.015318,-0.013930,-0.015276,0.002830,0.000745,0.000412,0.000077,0.000077,0.025472,0.072294,0.072128,0.002162,0.004485,-0.356854,-0.108553,-0.083858,-0.253355,-0.009850,0.009225,0.008473,0.009310,0.008648,0.009055,-0.04198,-0.003694,0.010298,0.004233,-0.0087,-0.02727,0.038598,-0.008306,0.024074
319107,-0.006886,-0.013205,-0.015521,-0.006897,-0.015427,0.001609,0.000855,0.000451,0.001054,-0.000311,0.014480,0.082920,0.078920,0.029500,-0.018029,-0.403751,-0.418091,-0.657603,0.363897,-0.352495,0.006004,0.008221,0.009067,0.005644,0.008834,-0.04198,-0.003694,0.010298,0.004233,-0.0087,-0.02727,0.038598,-0.008306,0.024074
319108,-0.018143,-0.011536,-0.012432,-0.015143,-0.013968,-0.001752,0.000752,0.000646,0.001086,-0.000156,-0.015770,0.072918,0.113009,0.030418,-0.009030,-0.652002,-0.272680,-0.287846,-0.548325,-0.145080,0.010819,0.007315,0.007044,0.008381,0.008065,-0.04198,-0.003694,0.010298,0.004233,-0.0087,-0.02727,0.038598,-0.008306,0.024074
319109,-0.007447,-0.010656,-0.012399,-0.007459,-0.011743,0.002182,0.000705,0.000522,0.001350,0.000189,0.019640,0.068421,0.091412,0.037794,0.010947,-0.182817,-0.279285,-0.489138,-0.317276,-0.186464,0.007347,0.006080,0.008297,0.006239,0.006732,-0.04198,-0.003694,0.010298,0.004233,-0.0087,-0.02727,0.038598,-0.008306,0.024074
319110,-0.002923,-0.009040,-0.012328,-0.009508,-0.010451,0.000889,0.000460,0.000198,0.000520,0.000273,0.008005,0.044612,0.034735,0.014567,0.015806,1.003276,-0.563710,-0.376788,-0.842795,-0.864175,0.003390,0.005434,0.007714,0.004763,0.005482,-0.04198,-0.003694,0.010298,0.004233,-0.0087,-0.02727,0.038598,-0.008306,0.024074


In [ ]:
# Sum the returns across all assets for each day to get the daily total return
daily_total_returns = ab.sum(axis=1)
cumulative_total_returns = daily_total_returns.cumsum()
import matplotlib.pyplot as plt

# Plotting the cumulative total returns
cumulative_total_returns.plot(figsize=(10, 6), title='Cumulative Total Returns Over Time')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.grid(True)
plt.show()



In [ ]:
cumulative_total_returns[-1]

In [ ]:
import matplotlib.pyplot as plt

# Plot daily total returns
plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
daily_total_returns.plot(title='Daily Total Returns')
plt.ylabel('Daily Return')

# Plot cumulative return
plt.subplot(1, 2, 2)
cumulative_returns = (1 + daily_total_returns).cumprod() - 1
cumulative_returns.plot(title='Cumulative Return')
plt.ylabel('Cumulative Return')

plt.show()


In [ ]:

model_returns = {}

for model, df_indicator in model_dfs.items():
    # Multiplying indicator DataFrame with returns DataFrame
    weighted_returns = df_indicator * df_2_read
    
    # Store the weighted returns DataFrame
    model_returns[model] = weighted_returns


